In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from collections import OrderedDict
import subprocess
import xml.etree.ElementTree as ET

In [2]:
#Converts valid split to image paths
valid_in = open("./TrainVal/VOCdevkit/VOC2011/ImageSets/Segmentation/val.txt", "r")
valid_out = open("./valid.txt", "w")
for line in valid_in:
    valid_out.write("../TrainVal/VOCdevkit/VOC2011/JPEGImages/" + line[:-1] + ".jpg\n")
valid_in.close()
valid_out.close()

In [3]:
#Runs shell script to install darknet files and run inference...takes a long time so the txt are submitted too
#This step is only included for reproducibility
# subprocess.run(["chmod", "+x", "install-dependencies.sh"])
# proc = subprocess.run("./install-dependencies.sh",  capture_output=True, shell=True)
# print(proc)

In [4]:
#Builds dataframe for use throughout the program
def build_df(annot_path, image_path, split_path):    
    data = pd.DataFrame(columns=["image", "annotation"])
    split = open(split_path, "r")
    lines = split.readlines()
    split.close()
    jpg = []
    xml = []
    for i in lines:
        temp = i.split(" ")[0][:-1]
        jpg.append(image_path+temp+".jpg")
        xml.append(annot_path + temp + ".xml")
    data["image"] = jpg
    data["annotation"] = xml
    return data
    


In [5]:
# Function to extract coordinates, labels, and bounding boxes from yolo predictions
#Data is the datframe
#File path is the file path oif the output of the models predictions
#Start index can either be 4 or 7 depending on the model (4 -> tiny, 6-> standard)
def extraction(data, file_path, start_ind):
    assert start_ind == 4 or start_ind == 6
    idx = 0
    images = OrderedDict()
    for path in data["image"]:
        images[path] = []
    fp = None
    times = []
    with open(file_path) as f:
        lines = f.readlines()
        for line in lines[start_ind:-1]:
            split = line.split()
            if split[0] == "Enter":
                fp = data["image"].iloc[idx]
                idx+=1
            elif split[0][0] == ".":
                times.append(split[3])
            else:
                box = [split[0], split[1][:-1], split[3], split[5], split[7], split[9][:-1]]
                images[fp].append(box)
    return images, times
                
                
                

In [6]:
#Parses through XML annotations and reads bb information
def find_boxes(file):

    tree = ET.parse(file)
    root = tree.getroot()

    boxes = []
    
    for box in root.iter('object'):
        name = str(box.find("name").text)
        ymin = int(box.find("bndbox/ymin").text)
        xmin = int(box.find("bndbox/xmin").text)
        ymax = int(box.find("bndbox/ymax").text)
        xmax = int(box.find("bndbox/xmax").text)
        width = xmax - xmin
        height = ymax - ymin
        boxes.append([name, xmin, ymin, width, height])
        
    return boxes



In [7]:
#Import the annotations to RAM using the above function
def import_annotations(data):
    counter = 0
    annotations = OrderedDict()
    for file in data["annotation"]:
        boxes = find_boxes(file)
        annotations[data["image"].iloc[counter]] = boxes
        counter+=1
    return annotations
    
    

In [8]:

def IoU(gt, pred):
    gt_x, gt_y, gt_w, gt_h = gt[1:]
    
    pred_x, pred_y, pred_w, pred_h = pred[2:]
    
    x_left = max(gt_x, pred_x)
    y_top = max(gt_y, pred_y)
    x_right = min((gt_x + gt_w), (pred_x + pred_w))
    y_bottom = min((gt_y+gt_h), (pred_y+pred_h))
    
    intersect = (x_right - x_left) * (y_bottom - y_top)
    ground_area = gt_w * gt_h
    pred_area = pred_w * pred_h
    iou = intersect / (ground_area + pred_area - intersect)
    return iou
    

In [9]:
def mAP_helper(gt, pred):
    output = None
    iou = IoU(gt, pred)
    if (iou >= 0.5 and gt[i][0] == None) or (iou >= 0.5 and gt[i][0] != pred[i][0]):
        output = "FN"
    elif iou >= 0.5:
        output = "TP"
    else:
        output = "FP"
    return output   

In [10]:
def tiny_yolo():
    annot_path = "./TrainVal/VOCdevkit/VOC2011/Annotations/"
    image_path = "./TrainVal/VOCdevkit/VOC2011/JPEGImages/"
    split_path = "./TrainVal/VOCdevkit/VOC2011/ImageSets/Segmentation/val.txt"
    data = build_df(annot_path, image_path, split_path)
    predictions, times = extraction(data, "tiny_results.txt", 4)
    ground_truth = import_annotations(data)
    conf_thresh = 70
    counter = 0
    
    
    with open("tiny_yolo_metrics.txt", "w") as f:
        for pred in predictions:
            f.write("Image Path: " + pred)
            f.write("\t\tTime for predictions: " + str(times[counter]) + "\n")
            counter+=1
    

In [11]:
tiny_yolo()

In [12]:
def standard_yolo():
    annot_path = "./TrainVal/VOCdevkit/VOC2011/Annotations/"
    image_path = "./TrainVal/VOCdevkit/VOC2011/JPEGImages/"
    split_path = "./TrainVal/VOCdevkit/VOC2011/ImageSets/Segmentation/val.txt"
    data = build_df(annot_path, image_path, split_path)
    predictions, times = extraction(data, "standard_results.txt", 6)
    ground_truth = import_annotations(data)
    conf_thresh = 70
    counter = 0
    
    
    with open("standard_yolo_metrics.txt", "w") as f:
        for pred in predictions:
            f.write("Image Path: " + pred)
            f.write("\t\tTime for predictions: " + str(times[counter]) + "\n")
            counter+=1

In [13]:
standard_yolo()